In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy import constants, units
from astropy.coordinates import Angle
from scipy.optimize import curve_fit
import pandas as pd
import os

In [2]:
rootdir = '/Users/thepoetoftwilight/Documents/CUBS/Data/PG1522+101/MUSE/'

Load in the segmentation map for galaxy IDs, coordinates, and redshifts

In [3]:
cat_file = np.loadtxt(rootdir + 'test_new_updated.cat')

In [4]:
# Load in the MUSE file
muse_fits = fits.open(rootdir + 'COMBINED_CUBE_MED_FINAL_vac.fits')
data_hdu = muse_fits[1]
spec_cube = data_hdu.data.copy()

In [5]:
# Load in the segmentation file
seg_fits = fits.open(rootdir+'test_seg.fits')
seg_map = seg_fits[0].data

Load in the necessary fields for all the galaxies within this catalog

In [6]:
# Galaxy IDs (can help extract the spectra)
gal_ids_arr = cat_file[:,0]

# Galaxy redshifts
gal_z_arr = cat_file[:,12]

In [7]:
# Galaxy coordinates, first load in degrees
gal_ra_arr_deg = cat_file[:,1]
gal_dec_arr_deg = cat_file[:,2]

gal_ra_arr_hms = [Angle(ra, units.degree).hms for ra in gal_ra_arr_deg]
gal_dec_arr_dms = [Angle(dec, units.degree).dms for dec in gal_dec_arr_deg]

gal_ra_arr_hms_str = np.array([str(int(ra[0])) + 'h' + str(int(ra[1])) + 'm' + str(np.round(ra[2],2)) + 's' 
                 for ra in gal_ra_arr_hms])

gal_dec_arr_dms_str = np.array([str(int(dec[0])) + 'd' + str(int(dec[1])) + 'm' + str(np.round(dec[2],2)) + 's' 
                 for dec in gal_dec_arr_dms])

In [8]:
# Galaxy coordinates in physical units
gal_x_arr = cat_file[:,3]
gal_y_arr = cat_file[:,4]

In [9]:
# Define a function to load in the spectrum of a given galaxy
def load_spec(rootdir, gal_id):
    
    # Construct a wavelength array
    wav_0 = data_hdu.header['CRVAL3']
    del_wav = data_hdu.header['CD3_3']
    wav_arr = np.arange(wav_0, wav_0 + (spec_cube.shape[0]-1)*del_wav, del_wav)
    
    # Isolate coordinates from segmentation map
    seg_map_gal_y, seg_map_gal_x = np.where(seg_map==gal_id)

    # Get bolometric fluxes for each pixel in the galaxy
    flux_gal = np.zeros(len(seg_map_gal_x))
    
    # Also isolate the spectra for each pixel
    spec_stack = np.zeros((len(seg_map_gal_x), len(wav_arr)))
    
    for j in range(len(seg_map_gal_x)):
        
        # Get the x and y coordinates
        x = seg_map_gal_x[j]
        y = seg_map_gal_y[j]

        # Get the bolometric flux for this coordinate
        flux_gal[j] = np.nansum(spec_cube[:,y,x])
        
        # Also isolate the spectrum for the pixel
        spec_stack[j,:] = spec_cube[:,y,x]
        
    # These are some bolometric flux cuts to include/ exclude pixels when stacking the spectra
    min_flux_gal = np.nanpercentile(flux_gal, 75)
    max_flux_gal = np.nanpercentile(flux_gal, 100)
    
    idx_bright = (flux_gal>=min_flux_gal) & (flux_gal<=max_flux_gal)
    
    # Undo normalization before saving the spectrum
    f_lam = np.nansum(spec_stack[idx_bright], axis=0)*1e-20
    
    return wav_arr, f_lam

From CUBS I, we have the following bandpasses -

<blockquote>4800–5800 Å (pseudo g-band), 6000–7000 Å (pseudo r-band), and 7500–8500 Å (pseudo i-band)</blockquote>

But in CUBS VI, we modify the bandpass to be -

<blockquote>4800-5800 Å (pseudo g-band), 6000-7500 Å (pseudo r-band), and 7500-9000 Å (pseudo i-band)</blockquote>

Let's try performing this calculation for all galaxies

In [10]:
# Define this for automation later
bandpasses_lam = {'pseudo_g': [4800,5800], 'pseudo_r': [6000,7500], 'pseudo_i': [7500,9000]}

In [11]:
# Define a function that, given a spectrum, can compute the AB magnitude in a given bandpass

def calc_AB_mag(wav, f_lam, bandpass):
    
    # First, slice up the spectrum to isolate points within the bandpass
    bandpass_mask = ((wav>bandpass[0])&(wav<bandpass[1]))
    
    wav_mask = wav[bandpass_mask]
    f_lam_mask = f_lam[bandpass_mask]
    
    # Then convert f_lam to f_nu
    
    # Conversion first converts ergs/cm^2/s/Å to W/m^2/m
    # Then converts wavelength in Å to wavelength in m
    # Then applies the speed of light in m/s
    # Then converts f_nu from W/m^2/Hz to Jy=
    f_nu_mask = wav_mask**2*f_lam_mask*(3.34e+4)
    
    # Also get the relevant frequencies (in Hz) in the bandpass
    c = 3e+8 # in m/s
    nu_mask = c/(wav_mask*1e-10)
 
        
    # Compute the magnitude
    AB_mag = -2.5*np.log10(np.trapz(y=f_nu_mask/nu_mask, x=nu_mask)/np.trapz(y=3631/nu_mask, x=nu_mask)) 
            
    return AB_mag

In [12]:
# Initalize magnitude arrays

gal_pseudo_g_arr = np.zeros(len(gal_ids_arr))
gal_pseudo_r_arr = np.zeros(len(gal_ids_arr))
gal_pseudo_i_arr = np.zeros(len(gal_ids_arr))

In [13]:
for i in range(len(gal_ids_arr)):
    
    # Load in the spectrum of the galaxy
    wav, f_lam = load_spec(rootdir, int(gal_ids_arr[i]))
    
    gal_pseudo_g_arr[i] = calc_AB_mag(wav, f_lam, bandpasses_lam['pseudo_g'])
    gal_pseudo_r_arr[i] = calc_AB_mag(wav, f_lam, bandpasses_lam['pseudo_r'])
    gal_pseudo_i_arr[i] = calc_AB_mag(wav, f_lam, bandpasses_lam['pseudo_i'])

Save everything in a neat catalog

In [14]:
with open(rootdir+'pseudo_gri_photometry.dat', 'w') as f:
    
    f.write('ID,RA,Dec,x,y,pseudo_g_mag,pseudo_r_mag,pseudo_i_mag,z')
    
    for i in range(len(gal_ids_arr)):
        
        f.write('\n'+str(int(gal_ids_arr[i])) + ',' + 
          str(gal_ra_arr_deg[i]) + ',' + 
          str(gal_dec_arr_deg[i]) + ',' +
          str(gal_x_arr[i]) + ',' +
          str(gal_y_arr[i]) + ',' +
          str('%.2f'%np.round(gal_pseudo_g_arr[i],2)) + ',' + 
          str('%.2f'%np.round(gal_pseudo_r_arr[i],2)) + ',' + 
          str('%.2f'%np.round(gal_pseudo_i_arr[i],2)) + ',' +
          str(gal_z_arr[i]))